# Step 1: Preprocessing the Data

In [1]:
# Step 1: Load & clean the dataset
import pandas as pd
import numpy as np

In [3]:
# Load TSV
df = pd.read_csv("train.tsv", sep="\t", header=None)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       10240 non-null  object 
 1   1       10240 non-null  object 
 2   2       10240 non-null  object 
 3   3       10238 non-null  object 
 4   4       10238 non-null  object 
 5   5       7342 non-null   object 
 6   6       8030 non-null   object 
 7   7       10238 non-null  object 
 8   8       10238 non-null  float64
 9   9       10238 non-null  float64
 10  10      10238 non-null  float64
 11  11      10238 non-null  float64
 12  12      10238 non-null  float64
 13  13      10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


In [7]:
# Column assignment (based on Liar dataset)
df.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 
              'state', 'party', 'barely_true_counts', 'false_counts', 'half_true_counts', 
              'mostly_true_counts', 'pants_on_fire_counts', 'context']

In [9]:
# Keep only statement and label
df = df[['statement', 'label']]

In [11]:
# Filter valid labels
valid_labels = ['true', 'false']
df = df[df['label'].isin(valid_labels)]

In [13]:
# Map labels to binary
df['label'] = df['label'].map({'true': 1, 'false': 0})

In [15]:
df.head()

,statement,label
0,Says the Annies List political group supports ...,0
3,Health care reform legislation is likely to ma...,0
5,The Chicago Bears have had more starting quart...,1
12,When Mitt Romney was governor of Massachusetts...,0
16,McCain opposed a requirement that the governme...,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3671 entries, 0 to 10238
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  3671 non-null   object
 1   label      3671 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 86.0+ KB


# Step 2: Tokenization with transformers

In [20]:
!pip install transformers datasets torch --quiet

In [22]:
# Step 2: Tokenization and Dataset Preparation
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch

In [24]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [25]:
# Tokenize the statements
tokens = tokenizer(
    list(df['statement']),
    max_length=128,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [28]:
# Labels
labels = torch.tensor(df['label'].values)

In [30]:
# Train-test split (80/20)
input_ids_train, input_ids_test, \
attention_mask_train, attention_mask_test, \
labels_train, labels_test = train_test_split(
    tokens['input_ids'], tokens['attention_mask'], labels,
    test_size=0.2, random_state=42
)

# Step 3: Load BERT and Prepare for Fine-tuning

In [33]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn

In [35]:
# Load pre-trained BERT with classification head (2 classes)
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [39]:
# Create DataLoaders
batch_size = 16

train_data = TensorDataset(input_ids_train, attention_mask_train, labels_train)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(input_ids_test, attention_mask_test, labels_test)
test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Step 4: Training Loop

In [42]:
from transformers import get_scheduler
import time
from tqdm import tqdm

In [44]:
# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3

In [46]:
num_training_steps = len(train_loader) * epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [48]:
# Training loop
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    total_loss = 0

    for batch in tqdm(train_loader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Average training loss: {avg_loss:.4f}")

Epoch 1/3


100%|████████████████████████████████████████████████████████████████████████████████| 184/184 [09:06<00:00,  2.97s/it]


Average training loss: 0.6729
Epoch 2/3


100%|████████████████████████████████████████████████████████████████████████████████| 184/184 [09:15<00:00,  3.02s/it]


Average training loss: 0.5997
Epoch 3/3


100%|████████████████████████████████████████████████████████████████████████████████| 184/184 [09:39<00:00,  3.15s/it]

Average training loss: 0.4538


# Step 5: Model Evaluation (Accuracy, F1 Score, Confusion Matrix)

In [108]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np

In [109]:
# Set model to evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [113]:
# Track predictions and true labels
all_preds = []
all_labels = []

In [115]:
with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=1).flatten()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [116]:
# Convert to NumPy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

In [117]:
# Accuracy and F1
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

In [118]:
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.6272
F1 Score: 0.6277

Confusion Matrix:
[[230 161]
 [113 231]]


In [119]:
# Detailed classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["False", "True"]))


Classification Report:
              precision    recall  f1-score   support

       False       0.67      0.59      0.63       391
        True       0.59      0.67      0.63       344

    accuracy                           0.63       735
   macro avg       0.63      0.63      0.63       735
weighted avg       0.63      0.63      0.63       735



# Step 6: Save the Trained Model

In [124]:
# Save the model and tokenizer to a local folder
model_save_path = "saved_bert_model1"

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('saved_bert_model1\\tokenizer_config.json',
 'saved_bert_model1\\special_tokens_map.json',
 'saved_bert_model1\\vocab.txt',
 'saved_bert_model1\\added_tokens.json')

# Step 7: Deploying with FastAPI

In [126]:
!pip install fastapi uvicorn transformers torch nest_asyncio --quiet

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Allow running FastAPI in Jupyter
nest_asyncio.apply()

# Load model and tokenizer
model_path = "saved_bert_model1"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Setup FastAPI
app = FastAPI()

class TextInput(BaseModel):
    text: str

@app.post("/predict")
async def predict(input: TextInput):
    inputs = tokenizer(
        input.text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    label = "true" if prediction == 1 else "false"
    return {"prediction": label}

# Run server inside notebook
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [4384]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


# Step 8: Integrate ChatGPT API

In [70]:
!pip install openai --quiet

In [95]:
import openai

# Set your API key
openai.api_key = "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"

def classify_with_chatgpt(statement):
    prompt = f"""You are a fact-checking assistant. Classify the following claim as either "true" or "false":

Claim: "{statement}"

Your response should be just one word: "true" or "false"."""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Or "gpt-4" if you want
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1,
            temperature=0
        )
        label = response['choices'][0]['message']['content'].strip().lower()
        return label if label in ['true', 'false'] else 'unknown'

    except Exception as e:
        print("Error:", e)
        return "error"


In [101]:
# Sample 5 rows from the dataframe
sample_df = df.sample(5)

# Loop through and classify each claim
for i, row in sample_df.iterrows():
    claim = row['statement']
    true_label = 'true' if row['label'] == 1 else 'false'

    prediction = classify_with_chatgpt(claim)
    print(f"Claim: {claim}\nTrue Label: {true_label} | ChatGPT: {prediction}\n")

Error: Incorrect API key provided: sk-or-v1*************************************************************611b. You can find your API key at https://platform.openai.com/account/api-keys.
Claim: Everything I have said (on the campaign trail) has been factually accurate.
True Label: false | ChatGPT: error

Error: Incorrect API key provided: sk-or-v1*************************************************************611b. You can find your API key at https://platform.openai.com/account/api-keys.
Claim: Mickey Mouse was registered to vote (in Florida).
True Label: false | ChatGPT: error

Error: Incorrect API key provided: sk-or-v1*************************************************************611b. You can find your API key at https://platform.openai.com/account/api-keys.
Claim: John McCain accused Barack Obama "of letting infants die."
True Label: false | ChatGPT: error

Error: Incorrect API key provided: sk-or-v1*************************************************************611b. You can find your API